In [6]:
from flask import Flask
import openai

In [7]:
import requests

In [8]:
app=Flask(__name__)

In [ ]:
open_ai_api_key=""
SYSCANNER_API_KEY = "your_skyscanner_api_key"
WEATHER_API_KEY = "your_weather_api_key"


In [1]:
# Function to get places of interest
def get_places(destination, place_type="tourist attractions"):
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={place_type}+in+{destination}&key={GOOGLE_PLACES_API_KEY}"
    response = requests.get(url)
    data = response.json()
    places = [place["name"] for place in data.get("results", [])[:5]]
    return places

In [2]:

# Function to get flight details
def get_flights(source, destination, date):
    url = f"https://partners.api.skyscanner.net/apiservices/browsequotes/v1.0/US/USD/en-US/{source}/{destination}/{date}"
    headers = {"apikey": SKYSCANNER_API_KEY}
    response = requests.get(url, headers=headers)
    return response.json()

In [3]:
# Function to get weather details
def get_weather(destination):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={destination}&appid={WEATHER_API_KEY}"
    response = requests.get(url)
    data = response.json()
    return f"{data['weather'][0]['description']}, Temp: {round(data['main']['temp'] - 273.15, 2)}°C"


In [9]:
# AI-generated itinerary
@app.route('/generate-itinerary', methods=['POST'])
def generate_itinerary():
    user_input = request.json
    destination = user_input.get("destination")
    budget = user_input.get("budget")
    interests = user_input.get("interests")

    # Fetch additional data
    places = get_places(destination)
    weather = get_weather(destination)

    # Generate AI itinerary
    prompt = f"""
    Create a 3-day travel itinerary for {destination}.
    Budget: {budget}. Interests: {interests}.
    Must include recommended places: {', '.join(places)}.
    Current weather: {weather}.
    """

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": prompt}]
    )

    itinerary = response.choices[0].message["content"]

    return jsonify({"destination": destination, "weather": weather, "itinerary": itinerary})

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)